In [1]:
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt

### Steps in Generating Demand Matrix

In [2]:
# CSV File generated from QGIS
df_centroids_to_node = pd.read_csv("sample_barangay_centroids_to_nearest_node.csv")
#df_centroids_to_node

# For sample only, will delete later
df_centroids_to_node.drop(index=[6], inplace=True)
#df_centroids_to_node

df_cn = df_centroids_to_node[["Barangay Name","HubName"]]
df_cn.set_index('Barangay Name', inplace=True)
nodes = list(df_cn["HubName"].unique())
centroids = list(df_cn.index)
df_cn_final = pd.DataFrame(nodes, columns=["Node"])

list_all = []
for node in nodes:
    list_c = []
    for centroid in centroids:
        a = df_cn.loc[centroid] == node
        if a.iloc[0]:
            list_c.append(centroid)
    list_all.append(list_c)
    
df_cn_final["Centroids"] = list_all
df_cn_final.set_index("Node", inplace=True)
df_cn_final

,Centroids
Node,
Sta. Lucia Grandmall,[San Roque]
Bayan Bayanan,"[Concepcion Dos, Concepcion Uno]"
Ligaya,[Calumpang]
Riverbanks,[Barangka]
Marikina Bayan,"[Jesus de La Pena, Santo Nino, Santa Elena]"
Tumana,"[Tumana, Marikina Heights]"
Nangka,"[Fortune, Nangka]"
Fairlane,[Parang]
Golden Valley,[Malanday]


#### 9 Modal Split
#### 9.1 Utility Matrices 

Centroid and then to Node: Cost Matrix Tables (in terms of time and cost) using QGIS

Network Type: https://www.qgistutorials.com/en/docs/3/downloading_osm_data.html

Vector -> QuickOSM -> QuickOSM dialog box

Processing -> Toolbox -> QNEAT3 -> Distance Matrices -> OD-Matrix From Points as Table (n:n)

In [3]:
# Travel Time Matrix, Travel Cost Matrix
# Bus, Car, Motorcycle

# bus: 15 pesos for the first 5 km, 2 pesos for every succeeding km

In [4]:
csv_bus_travel_dist_mat = pd.read_csv("cost_matrix.csv")
csv_bus_travel_dist_mat.head()

,origin_id,destination_id,entry_cost,network_cost,exit_cost,total_cost
0,Barangka,Barangka,0.000000,0.000000,0.000000,0.000000
1,Barangka,Calumpang,0.000016,3216.298944,0.000454,3216.299414
2,Barangka,Concepcion Dos,0.000016,5861.484544,0.003020,5861.487580
3,Barangka,Concepcion Uno,0.000016,3447.175599,0.000032,3447.175646
4,Barangka,Fortune,0.000016,5929.364535,0.009198,5929.373749


In [5]:
# Convert to a Matrix
df_bus_travel_dist_mat = csv_bus_travel_dist_mat.drop(columns=["entry_cost","network_cost","exit_cost"])
df_bus_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,Barangka,Barangka,0.000000
1,Barangka,Calumpang,3216.299414
2,Barangka,Concepcion Dos,5861.487580
3,Barangka,Concepcion Uno,3447.175646
4,Barangka,Fortune,5929.373749


In [6]:
# assign centroid to a centroid_number
list_cen = df_bus_travel_dist_mat["origin_id"].unique()
df_cen_no = pd.DataFrame(list_cen, columns=["Centroid"])
cen_no = list(range(0,len(list_cen)))
df_cen_no["Number"] = cen_no
df_cen_no.set_index("Centroid", inplace=True)
df_cen_no.head()

,Number
Centroid,
Barangka,0
Calumpang,1
Concepcion Dos,2
Concepcion Uno,3
Fortune,4


In [7]:
def cen_number(centroid):
    number = df_cen_no["Number"][centroid]
    return number

In [8]:
df_bus_travel_dist_mat["origin_id"] = df_bus_travel_dist_mat["origin_id"].apply(cen_number)
df_bus_travel_dist_mat["destination_id"] = df_bus_travel_dist_mat["destination_id"].apply(cen_number)

In [9]:
df_bus_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,0,0,0.000000
1,0,1,3216.299414
2,0,2,5861.487580
3,0,3,3447.175646
4,0,4,5929.373749


In [10]:
rows = df_bus_travel_dist_mat["origin_id"].nunique()
cols = rows
 
dist_mat = [[0 for _ in range(cols)] for _ in range(rows)]

for a in range(0, len(df_bus_travel_dist_mat)):
    i = df_bus_travel_dist_mat["origin_id"][a]
    j = df_bus_travel_dist_mat["destination_id"][a]
    dist_mat[i][j] = df_bus_travel_dist_mat["total_cost"][a]

print(f'matrix of dimension {rows} x {cols} is {dist_mat}')

matrix of dimension 15 x 15 is [[0.0, 3216.2994136, 5861.4875803, 3447.1756463, 5929.3737488, 1036.4434799, 2896.572854, 4849.6456005, 6132.3279255, 5208.3365261, 2644.0540638, 2053.656446, 1713.7822483, 531.6994438, 4106.415622], [3216.2994136, 0.0, 6253.5320417, 3839.2201078, 6321.4182102, 2179.8569547, 3288.6173155, 5241.690062, 6524.372387, 5600.3809875, 886.1458906, 1486.4232269, 1816.4183854, 2684.6000048, 4498.4600835], [5861.4875803, 6253.5320417, 0.0, 2414.3119981, 4597.9158201, 4825.0451213, 2964.921968, 1011.8519305, 4800.8699968, 3876.8785974, 5681.2866919, 5090.8890741, 4751.0148764, 5329.7881715, 2774.9576934], [3447.1756463, 3839.2201078, 2414.3119981, 0.0, 2482.1981666, 2410.7331874, 550.6100341, 1402.4700183, 2685.1523433, 1761.1609439, 3266.9747579, 2676.5771402, 2336.7029425, 2915.4762375, 659.2400398], [5929.3737488, 6321.4182102, 4597.9158201, 2482.1981666, 0.0, 4892.9312899, 3032.8081365, 3586.0738404, 3725.5913484, 721.0372662, 5749.1728604, 5158.7752426, 4818.90

In [11]:
df_dist_mat = pd.DataFrame(dist_mat)
df_dist_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,3216.299414,5861.487580,3447.175646,5929.373749,1036.443480,2896.572854,4849.645601,6132.327925,5208.336526,2644.054064,2053.656446,1713.782248,531.699444,4106.415622
1,3216.299414,0.000000,6253.532042,3839.220108,6321.418210,2179.856955,3288.617316,5241.690062,6524.372387,5600.380987,886.145891,1486.423227,1816.418385,2684.600005,4498.460084
2,5861.487580,6253.532042,0.000000,2414.311998,4597.915820,4825.045121,2964.921968,1011.851930,4800.869997,3876.878597,5681.286692,5090.889074,4751.014876,5329.788172,2774.957693
3,3447.175646,3839.220108,2414.311998,0.000000,2482.198167,2410.733187,550.610034,1402.470018,2685.152343,1761.160944,3266.974758,2676.577140,2336.702943,2915.476238,659.240040
4,5929.373749,6321.418210,4597.915820,2482.198167,0.000000,4892.931290,3032.808136,3586.073840,3725.591348,721.037266,5749.172860,5158.775243,4818.901045,5397.674340,1822.970290
5,1036.443480,2179.856955,4825.045121,2410.733187,4892.931290,0.000000,1860.130395,3813.203142,5095.885467,4171.894067,1607.611605,1017.213987,677.339789,504.744071,3069.973163
6,2896.572854,3288.617316,2964.921968,550.610034,3032.808136,1860.130395,0.000000,1953.079988,3235.762313,2311.770914,2716.371966,2125.974348,1786.100150,2364.873445,1209.850010
7,4849.645601,5241.690062,1011.851930,1402.470018,3586.073840,3813.203142,1953.079988,0.000000,3789.028017,2865.036618,4669.444712,4079.047094,3739.172897,4317.946192,1763.115714
8,6132.327925,6524.372387,4800.869997,2685.152343,3725.591348,5095.885467,3235.762313,3789.028017,0.000000,3004.554126,5952.127037,5361.729419,5021.855222,5600.628517,2025.924466
9,5208.336526,5600.380987,3876.878597,1761.160944,721.037266,4171.894067,2311.770914,2865.036618,3004.554126,0.000000,5028.135638,4437.738020,4097.863822,4676.637117,1101.933067


In [12]:
#for g in range(0, len(df_cost_mat)):
#    df_cost_mat.loc[g,g] = 300

#df_cost_mat

In [13]:
# Convert from total_cost (distance) to minute

def dist_to_min(dist):
    fixed_rate = 25    # kph
    dist = dist/1000   # convert to km
    min = (dist/fixed_rate)*60
    return min

df_bus_travel_time_mat = pd.DataFrame(df_dist_mat.apply(dist_to_min))
df_bus_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,7.719119,14.067570,8.273222,14.230497,2.487464,6.951775,11.639149,14.717587,12.500008,6.345730,4.928775,4.113077,1.276079,9.855397
1,7.719119,0.000000,15.008477,9.214128,15.171404,5.231657,7.892682,12.580056,15.658494,13.440914,2.126750,3.567416,4.359404,6.443040,10.796304
2,14.067570,15.008477,0.000000,5.794349,11.034998,11.580108,7.115813,2.428445,11.522088,9.304509,13.635088,12.218134,11.402436,12.791492,6.659898
3,8.273222,9.214128,5.794349,0.000000,5.957276,5.785760,1.321464,3.365928,6.444366,4.226786,7.840739,6.423785,5.608087,6.997143,1.582176
4,14.230497,15.171404,11.034998,5.957276,0.000000,11.743035,7.278740,8.606577,8.941419,1.730489,13.798015,12.381061,11.565363,12.954418,4.375129
5,2.487464,5.231657,11.580108,5.785760,11.743035,0.000000,4.464313,9.151688,12.230125,10.012546,3.858268,2.441314,1.625615,1.211386,7.367936
6,6.951775,7.892682,7.115813,1.321464,7.278740,4.464313,0.000000,4.687392,7.765830,5.548250,6.519293,5.102338,4.286640,5.675696,2.903640
7,11.639149,12.580056,2.428445,3.365928,8.606577,9.151688,4.687392,0.000000,9.093667,6.876088,11.206667,9.789713,8.974015,10.363071,4.231478
8,14.717587,15.658494,11.522088,6.444366,8.941419,12.230125,7.765830,9.093667,0.000000,7.210930,14.285105,12.868151,12.052453,13.441508,4.862219
9,12.500008,13.440914,9.304509,4.226786,1.730489,10.012546,5.548250,6.876088,7.210930,0.000000,12.067526,10.650571,9.834873,11.223929,2.644639


In [14]:
no_of_centroids = len(df_dist_mat.index)
r = no_of_centroids
c = no_of_centroids

first_five = 15  
every_km = 2    
     
final_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        dist = df_dist_mat[i][j]
        dist = dist/1000     # convert to km

        if (dist <= 5):
            rate = first_five
        else:
            rate = first_five + every_km * (dist - 5) 
        
        final_cost_mat[i][j] = round(rate,2)

print(f'matrix of dimension {r} x {c} is {final_cost_mat}')

matrix of dimension 15 x 15 is [[15, 15, 16.72, 15, 16.86, 15, 15, 15, 17.26, 15.42, 15, 15, 15, 15, 15], [15, 15, 17.51, 15, 17.64, 15, 15, 15.48, 18.05, 16.2, 15, 15, 15, 15, 15], [16.72, 17.51, 15, 15, 15, 15, 15, 15, 15, 15, 16.36, 15.18, 15, 15.66, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [16.86, 17.64, 15, 15, 15, 15, 15, 15, 15, 15, 16.5, 15.32, 15, 15.8, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15.19, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15.48, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [17.26, 18.05, 15, 15, 15, 15.19, 15, 15, 15, 15, 16.9, 15.72, 15.04, 16.2, 15], [15.42, 16.2, 15, 15, 15, 15, 15, 15, 15, 15, 15.06, 15, 15, 15, 15], [15, 15, 16.36, 15, 16.5, 15, 15, 15, 16.9, 15.06, 15, 15, 15, 15, 15], [15, 15, 15.18, 15, 15.32, 15, 15, 15, 15.72, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15.04, 15, 15, 15, 15, 15, 15], [15, 15, 15.66, 15, 15.8, 15, 15, 15, 16.2, 15, 15, 1

In [15]:
df_bus_travel_cost_mat = pd.DataFrame(final_cost_mat)
df_bus_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,15.00,15.00,16.72,15,16.86,15.00,15,15.00,17.26,15.42,15.00,15.00,15.00,15.00,15
1,15.00,15.00,17.51,15,17.64,15.00,15,15.48,18.05,16.20,15.00,15.00,15.00,15.00,15
2,16.72,17.51,15.00,15,15.00,15.00,15,15.00,15.00,15.00,16.36,15.18,15.00,15.66,15
3,15.00,15.00,15.00,15,15.00,15.00,15,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15
4,16.86,17.64,15.00,15,15.00,15.00,15,15.00,15.00,15.00,16.50,15.32,15.00,15.80,15
5,15.00,15.00,15.00,15,15.00,15.00,15,15.00,15.19,15.00,15.00,15.00,15.00,15.00,15
6,15.00,15.00,15.00,15,15.00,15.00,15,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15
7,15.00,15.48,15.00,15,15.00,15.00,15,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15
8,17.26,18.05,15.00,15,15.00,15.19,15,15.00,15.00,15.00,16.90,15.72,15.04,16.20,15
9,15.42,16.20,15.00,15,15.00,15.00,15,15.00,15.00,15.00,15.06,15.00,15.00,15.00,15


In [16]:
df_cost = df_cn_final["Centroids"].to_frame()
list_c = df_cost["Centroids"].to_list()

list_cn = []
for c in list_c:
    ccn = []
    for d in c:
        cn = cen_number(d)
        ccn.append(cn)
    list_cn.append(ccn)
        
df_cost["Centroid_Numbers"] = list_cn
df_cost

,Centroids,Centroid_Numbers
Node,,
Sta. Lucia Grandmall,[San Roque],[10]
Bayan Bayanan,"[Concepcion Dos, Concepcion Uno]","[2, 3]"
Ligaya,[Calumpang],[1]
Riverbanks,[Barangka],[0]
Marikina Bayan,"[Jesus de La Pena, Santo Nino, Santa Elena]","[5, 12, 11]"
Tumana,"[Tumana, Marikina Heights]","[14, 7]"
Nangka,"[Fortune, Nangka]","[4, 8]"
Fairlane,[Parang],[9]
Golden Valley,[Malanday],[6]


In [17]:
# TENTATIVE (for sample only)
list_node_numbers = range(0,len(df_cost.index))
list_node_numbers

range(0, 10)

In [18]:
df_cost["Node_Number"] = list_node_numbers
df_cost

,Centroids,Centroid_Numbers,Node_Number
Node,,,
Sta. Lucia Grandmall,[San Roque],[10],0
Bayan Bayanan,"[Concepcion Dos, Concepcion Uno]","[2, 3]",1
Ligaya,[Calumpang],[1],2
Riverbanks,[Barangka],[0],3
Marikina Bayan,"[Jesus de La Pena, Santo Nino, Santa Elena]","[5, 12, 11]",4
Tumana,"[Tumana, Marikina Heights]","[14, 7]",5
Nangka,"[Fortune, Nangka]","[4, 8]",6
Fairlane,[Parang],[9],7
Golden Valley,[Malanday],[6],8


In [19]:
df_nn_cn = df_cost[["Node_Number","Centroid_Numbers"]].set_index("Node_Number")
df_nn_cn

,Centroid_Numbers
Node_Number,
0,[10]
1,"[2, 3]"
2,[1]
3,[0]
4,"[5, 12, 11]"
5,"[14, 7]"
6,"[4, 8]"
7,[9]
8,[6]


In [20]:
no_of_nodes = len(df_nn_cn.index)
no_of_nodes

10

In [21]:
df_bus_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,7.719119,14.067570,8.273222,14.230497,2.487464,6.951775,11.639149,14.717587,12.500008,6.345730,4.928775,4.113077,1.276079,9.855397
1,7.719119,0.000000,15.008477,9.214128,15.171404,5.231657,7.892682,12.580056,15.658494,13.440914,2.126750,3.567416,4.359404,6.443040,10.796304
2,14.067570,15.008477,0.000000,5.794349,11.034998,11.580108,7.115813,2.428445,11.522088,9.304509,13.635088,12.218134,11.402436,12.791492,6.659898
3,8.273222,9.214128,5.794349,0.000000,5.957276,5.785760,1.321464,3.365928,6.444366,4.226786,7.840739,6.423785,5.608087,6.997143,1.582176
4,14.230497,15.171404,11.034998,5.957276,0.000000,11.743035,7.278740,8.606577,8.941419,1.730489,13.798015,12.381061,11.565363,12.954418,4.375129
5,2.487464,5.231657,11.580108,5.785760,11.743035,0.000000,4.464313,9.151688,12.230125,10.012546,3.858268,2.441314,1.625615,1.211386,7.367936
6,6.951775,7.892682,7.115813,1.321464,7.278740,4.464313,0.000000,4.687392,7.765830,5.548250,6.519293,5.102338,4.286640,5.675696,2.903640
7,11.639149,12.580056,2.428445,3.365928,8.606577,9.151688,4.687392,0.000000,9.093667,6.876088,11.206667,9.789713,8.974015,10.363071,4.231478
8,14.717587,15.658494,11.522088,6.444366,8.941419,12.230125,7.765830,9.093667,0.000000,7.210930,14.285105,12.868151,12.052453,13.441508,4.862219
9,12.500008,13.440914,9.304509,4.226786,1.730489,10.012546,5.548250,6.876088,7.210930,0.000000,12.067526,10.650571,9.834873,11.223929,2.644639


In [22]:
r = no_of_nodes
c = no_of_nodes
 
final_node_time_mat = [[0 for _ in range(c)] for _ in range(r)]
final_node_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        list_i = df_nn_cn["Centroid_Numbers"][i]
        list_j = df_nn_cn["Centroid_Numbers"][j]

        sum_cost = 0
        sum_time = 0
        counter = 0
        for a in list_i:
            for b in list_j:
                sum_time = sum_time + df_bus_travel_time_mat[a][b]
                sum_cost = sum_cost + df_bus_travel_cost_mat[a][b]
                counter = counter + 1
        
        final_node_time_mat[i][j] = round(sum_time/counter,2)
        final_node_cost_mat[i][j] = round(sum_cost/counter,2)

print(f'travel time matrix of dimension {r} x {c} is {final_node_time_mat}')
print(f'travel cost matrix of dimension {r} x {c} is {final_node_cost_mat}')

travel time matrix of dimension 10 x 10 is [[0.0, 10.74, 2.13, 6.35, 2.51, 10.31, 14.04, 12.07, 6.52, 5.07], [10.74, 2.9, 12.11, 11.17, 8.84, 3.51, 8.74, 6.77, 4.22, 9.89], [2.13, 12.11, 0.0, 7.72, 4.39, 11.69, 15.41, 13.44, 7.89, 6.44], [6.35, 11.17, 7.72, 0.0, 3.84, 10.75, 14.47, 12.5, 6.95, 1.28], [2.51, 8.84, 4.39, 3.84, 1.09, 8.41, 12.14, 10.17, 4.62, 2.57], [10.31, 3.51, 11.69, 10.75, 8.41, 2.12, 6.73, 4.76, 3.8, 9.47], [14.04, 8.74, 15.41, 14.47, 12.14, 6.73, 4.47, 4.47, 7.52, 13.2], [12.07, 6.77, 13.44, 12.5, 10.17, 4.76, 4.47, 0.0, 5.55, 11.22], [6.52, 4.22, 7.89, 6.95, 4.62, 3.8, 7.52, 5.55, 0.0, 5.68], [5.07, 9.89, 6.44, 1.28, 2.57, 9.47, 13.2, 11.22, 5.68, 0.0]]
travel cost matrix of dimension 10 x 10 is [[15.0, 15.68, 15.0, 15.0, 15.0, 15.0, 16.7, 15.06, 15.0, 15.0], [15.68, 15.0, 16.26, 15.86, 15.03, 15.0, 15.0, 15.0, 15.0, 15.33], [15.0, 16.26, 15.0, 15.0, 15.0, 15.24, 17.84, 16.2, 15.0, 15.0], [15.0, 15.86, 15.0, 15.0, 15.0, 15.0, 17.06, 15.42, 15.0, 15.0], [15.0, 15.03

In [23]:
bus_travel_time_mat = pd.DataFrame(final_node_time_mat)
bus_travel_time_mat

,0,1,2,3,4,5,6,7,8,9
0,0.00,10.74,2.13,6.35,2.51,10.31,14.04,12.07,6.52,5.07
1,10.74,2.90,12.11,11.17,8.84,3.51,8.74,6.77,4.22,9.89
2,2.13,12.11,0.00,7.72,4.39,11.69,15.41,13.44,7.89,6.44
3,6.35,11.17,7.72,0.00,3.84,10.75,14.47,12.50,6.95,1.28
4,2.51,8.84,4.39,3.84,1.09,8.41,12.14,10.17,4.62,2.57
5,10.31,3.51,11.69,10.75,8.41,2.12,6.73,4.76,3.80,9.47
6,14.04,8.74,15.41,14.47,12.14,6.73,4.47,4.47,7.52,13.20
7,12.07,6.77,13.44,12.50,10.17,4.76,4.47,0.00,5.55,11.22
8,6.52,4.22,7.89,6.95,4.62,3.80,7.52,5.55,0.00,5.68
9,5.07,9.89,6.44,1.28,2.57,9.47,13.20,11.22,5.68,0.00


In [24]:
bus_travel_cost_mat = pd.DataFrame(final_node_cost_mat)
bus_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9
0,15.00,15.68,15.00,15.00,15.00,15.00,16.70,15.06,15.0,15.00
1,15.68,15.00,16.26,15.86,15.03,15.00,15.00,15.00,15.0,15.33
2,15.00,16.26,15.00,15.00,15.00,15.24,17.84,16.20,15.0,15.00
3,15.00,15.86,15.00,15.00,15.00,15.00,17.06,15.42,15.0,15.00
4,15.00,15.03,15.00,15.00,15.00,15.00,15.21,15.00,15.0,15.00
5,15.00,15.00,15.24,15.00,15.00,15.00,15.00,15.00,15.0,15.00
6,16.70,15.00,17.84,17.06,15.21,15.00,15.00,15.00,15.0,16.00
7,15.06,15.00,16.20,15.42,15.00,15.00,15.00,15.00,15.0,15.00
8,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.0,15.00
9,15.00,15.33,15.00,15.00,15.00,15.00,16.00,15.00,15.0,15.00
